# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski

### Instructions
- Sign up for RunPod here: https://runpod.io/?ref=n8yfwyum
    - Note: That's my personal referral link. Please don't use it if we are mortal enemies.

- Click *Deploy* on either `SECURE CLOUD` or `COMMUNITY CLOUD`

- Follow the rest of the instructions in this video: https://www.youtube.com/watch?v=7m__xadX0z0#t=5m33.1s

Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
# If running on Vast.AI, copy the code in this cell into a new notebook. Run it, then launch the `dreambooth_runpod_joepenna.ipynb` notebook from the jupyter interface.
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [ ]:
# BUILD ENV
from IPython.display import clear_output

try:
 !pip install omegaconf
 !pip install einops
 !pip install pytorch-lightning==1.6.5
 !pip install test-tube
 !pip install transformers
 !pip install kornia
 !pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
 !pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
 !pip install setuptools==59.5.0
 !pip install pillow==9.0.1
 !pip install torchmetrics==0.6.0
 !pip install -e .
 !pip install protobuf==3.20.1
 !pip install gdown
 !pip install -qq diffusers["training"]==0.3.0 transformers ftfy
 !pip install huggingface_hub
 !pip install captionizer==1.0.1
 !pip install ipywidgets

 clear_output()
 print("✅ Dependencies successfully installed")

except:
 print("❌ Error installing dependencies")



# Download the 1.5 model from hugging face
You can also provide your own v1.* model for training by uploading it and renaming it to "model.ckpt".  It should be in the same directory as dreambooth_runpod_joepenna.ipynb

In [ ]:
# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

## Dreambooth Training Environment Setup

### Training Images
WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

### Regularization Images
We've created the following image sets

`man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
`man_unsplash` - pictures from various photographers
`300_person_ddim`
`person_ddim`
`woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0
`person_ddim` is recommended

### Config

These do what they say.  Eventually we'll integrate this into the config.

* "v1-finetune_unfrozen.yaml"
* "v1-finetune_unfrozen_save_checkpoints_every_250_steps.yaml"
* "v1-finetune_unfrozen_save_checkpoints_every_500_steps.yaml"

In [ ]:
# Setup your Dreambooth Training Environment
%run juptyer-notebook-helpers/create-config-form-inputs.ipynb

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
import os
import sys
import json

# parse the config file "joepenna-dreambooth-configs/active-config.json"
config_file_path = "./joepenna-dreambooth-configs/active-config.json";
if not os.path.exists(config_file_path):
 print(f"{config_file_path} not found.", file=sys.stderr)
else:
 config_file = open(config_file_path)
 config_parsed = json.load(config_file)
 print("✅ Loaded the config file, proceeding to training")
 print()
 print("active-config.json")
 print("------------------")
 print(json.dumps(config_parsed, indent=4))


 flip_p_arg = 0.5
 if config_parsed.face_training:
  flip_p_arg = 0.0

 reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + config_parsed.dataset

 !rm -rf training_images/.ipynb_checkpoints

 !python "main.py" \
     --base "configs/stable-diffusion/{config_parsed.config}" \
     -t \
     --actual_resume "model.ckpt" \
     --reg_data_root "{reg_data_root}" \
     -n "{config_parsed.project_name}" \
     --gpus 0, \
     --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
     --max_training_steps {config_parsed.max_training_steps} \
     --class_word "{config_parsed.class_word}" \
     --token "{config_parsed.token}" \
     --no-test \
     --flip_p {flip_p_arg}

## Copy and name the checkpoint file(s)

In [ ]:
import re

# parse the config file "joepenna-dreambooth-configs/active-config.json"
config_file_path = "./joepenna-dreambooth-configs/active-config.json";
if not os.path.exists(config_file_path):
 print(f"{config_file_path} not found.", file=sys.stderr)
else:
 config_file = open(config_file_path)
 config_parsed = json.load(config_file)
 training_images = !find training_images/*
 date_string = !date +"%Y-%m-%dT%H-%M-%S"

 if config_parsed.config == "v1-finetune_unfrozen.yaml":
  # Copy the checkpoint into our `trained_models` folder
  directory_paths = !ls -d logs/*
  last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
  file_name = date_string[-1] + "_" + \
              config_parsed.project_name + "_" + \
              str(len(training_images)) + "_training_images_" + \
              str(config_parsed.max_training_steps) + "_max_training_steps_" + \
              config_parsed.token + "_token_" + \
              config_parsed.class_word + "_class_word.ckpt"

  file_name = file_name.replace(" ", "_")

  !mkdir -p trained_models
  !mv "{last_checkpoint_file}" "trained_models/{file_name}"

  print("Download your trained model from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")
 else:
  directory_paths = !ls -d logs/*
  checkpoints_directory = directory_paths[-1] + "/checkpoints/trainstep_checkpoints"
  file_paths = !ls -d "{checkpoints_directory}"/*

  for i, original_file_name in enumerate(file_paths):
   # Remove the "epoch=000000-step=0000" text
   steps = re.sub(checkpoints_directory + "/epoch=\d{6}-step=0*", "", original_file_name)

   # Remove the .ckpt
   steps = steps.replace(".ckpt", "")

   # Setup the filename
   file_name = date_string[-1] + "_" + \
                   config_parsed.project_name + "_" + \
                   str(len(training_images)) + "_training_images_" + \
                   steps + "_training_steps_" + \
                   config_parsed.token + "_token_" + \
                   config_parsed.class_word + "_class_word.ckpt"

   file_name = file_name.replace(" ", "_")

   # Make the directory and move the files into it.
   !mkdir -p trained_models
   !mv "{original_file_name}" "trained_models/{file_name}"

  print("Download your trained models from the 'trained_models' folder and use in your favorite Stable Diffusion repo!")

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`